##  Viết nhanh level -1

In [11]:
import os
game_name = 'WelcomeToTheDungeon_v1'
start = ',\n  "-1": [\n   [],\n   [],\n   ['
print(start)
for i in os.listdir(f'C:\Users\admin\ENV\src\Agent\Ifelse\WelcomeToTheDungeon_v1'):
    print(f'    "{i[:-3]}"', end=',\n')
end = '   ]\n  ]'
print(end)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3873961764.py, line 5)

In [12]:
import json
a = json.load(open('A:\ENV\src\Log\level_game.json'))
for i in a:
    print(i, '-------------', list(a[i].keys())[2:])

FileNotFoundError: [Errno 2] No such file or directory: 'A:\\ENV\\Log\\level_game.json'

##  Kiểm tra hiệu suất của mô hình

In [1]:
%cd ..

/Users/mac/Monthy/ENV/tests


In [2]:
%cd ..

/Users/mac/Monthy/ENV


In [3]:
##  Kiểm tra hiện suất của hệ thống
#  !git clone https://github.com/ngoxuanphong/ENV.git
import warnings 
warnings.filterwarnings('ignore')
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning,NumbaExperimentalFeatureWarning, NumbaWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaExperimentalFeatureWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

from numba import njit
from numba.typed import List
import numpy as np
import env
import os
import importlib
import pandas as pd
from time import time

In [4]:
def test_game(game_name, dict_test = {}):
    importlib.reload(env)
    env.make(game_name)
    @njit()
    def Test(state, perData):
        validActions =env.getValidActions(state)
        arr_action = np.where(validActions==1)[0]
        idx = np.random.randint(0, arr_action.shape[0])
        return arr_action[idx], perData
    
    per_test = [0]
    a = time()
    win1, per_test = env.run(Test, num_game = 10000, agent_model = per_test, level = 0)
    b = time()
    try:
        win2, per_test = env.run(Test, num_game = 10000, agent_model = per_test, level = 1)
    except:
        win2 = False
    c = time()
    print(f'Game name: {game_name} | Win rate: ')
    print(f'        Level 0: {win1}/10000 in {int(b-a)} seconds')
    print(f'        Level 1: {win2}/10000 in {int(c-b)} seconds')
    try:
        win3, per_test = env.run(Test, num_game = 10000, agent_model = per_test, level = -1)
        d = time()
        time_lv_if = int(d-c)
    except:
        win3 = False
        time_lv_if = False
    if not win3:
        print(f'        Level -1: Chưa có level này')
    else:
        print(f'        Level -1: {win3}/10000 in {int(d-c)} seconds')
    dict_test[f"[{game_name}](https://github.com/ngoxuanphong/ENV/tree/main/Base/{game_name})"] = {
                            'State size': env.getStateSize(),
                            'Action size': env.getActionSize(),
                            'Win rate': [win1, win2, win3], 
                            'Time': [int(b-a), int(c-b), time_lv_if],
                            'Graphics': os.path.exists(f'src/Base/{game_name}/_render_func.py')}
    return dict_test

# test_game('MachiKoro')
# test_game('Splendor_v2')

In [5]:
list_game = os.listdir('src/Base')
dict_test = {}
for game_name in sorted(os.listdir('src/Base')):
    if '__' not in game_name:
        print(game_name)
        dict_test = test_game(game_name, dict_test)
# df = pd.DataFrame(dict_test).T
# df.to_csv('A:/ENV/Benchmark/time_win.csv')

Catan
Game name: Catan | Win rate: 
        Level 0: 2436/10000 in 192 seconds
        Level 1: 348/10000 in 166 seconds
        Level -1: 9/10000 in 121 seconds
CatanNoExchange
Game name: CatanNoExchange | Win rate: 
        Level 0: 2533/10000 in 135 seconds
        Level 1: 451/10000 in 671 seconds
        Level -1: 21/10000 in 77 seconds
Century
Game name: Century | Win rate: 
        Level 0: 1952/10000 in 27 seconds
        Level 1: 19/10000 in 40 seconds
        Level -1: Chưa có level này
Durak
Game name: Durak | Win rate: 
        Level 0: 2415/10000 in 12 seconds
        Level 1: 465/10000 in 17 seconds
        Level -1: Chưa có level này
Exploding_Kitten
Game name: Exploding_Kitten | Win rate: 
        Level 0: 1940/10000 in 13 seconds
        Level 1: 1716/10000 in 14 seconds
        Level -1: Chưa có level này
Fantan
Game name: Fantan | Win rate: 
        Level 0: 2452/10000 in 18 seconds
        Level 1: 114/10000 in 43 seconds
        Level -1: Chưa có level này
GoFish
G

##  Test lại agent level 2

In [ ]:
#  Test lại agent lv2

def test_game(game_name):
    env = make(game_name)
    getActionSize = env.getActionSize
    getValidActions = env.getValidActions
    getReward = env.getReward
    run = env.run

    @njit
    def Test(state,per):    
        actions =env.getValidActions(state)
        if per[-1][0][0][0][0] == 2:
            depth_tree = int(per[14][0][0][0][0])
            actions =env.getValidActions(state)
            output = per[-4][0][0][depth_tree] * actions + actions
            action = np.argmax(output)
        else:
            weight = per[-11][0][0][0]
            output = actions*weight + actions
            c = np.where(output == np.max(output))[0]
            action = np.random.choice(c)
            win = getReward(state)

            per[-11][0][0] += per[-9][0][0][action]
            if win != -1:
                per[-11][0][0] = np.zeros((1, getActionSize()))
        return action,per

    def convert_to_save(per):
        lst = []
        for i in range(len(per)):
            if len(per[i]) == 0:
                lst.append(List([np.zeros((1,1,1))]))
            else:
                lst.append(per[i])
        return lst

    def convert_to_test(lst):
        per = []
        for i in range(len(lst)):
            per.append(List(lst[i]))
        return per
    
    try:
        per_test = convert_to_test(np.load(f'A:\Trash\Data\{game_name}.npy', allow_pickle=True))
        win1, per_test = run(Test, 10000, per_test, 1)
        win2, per_test = run(Test, 10000, per_test, -1)
        print(f'Game name: {game_name} | Win rate: \n      Level 1: {win1}/10000 | Level -1: {win2}/10000')
    except:
        print(f'Game name: {game_name} | Game này chưa có level if else')

test_game('SushiGo')
for game_name in ['Splendor', 'TLMN', 'Catan']:
    test_game(game_name)
import os
list_game = os.listdir('A:\Trash\Data')
for game_name in list_game:
    test_game(game_name[:-4])

#  Test lại agent if else

In [2]:
##  Kiểm tra hiện suất của hệ thống
#  !git clone https://github.com/ngoxuanphong/ENV.git
%cd A:/ENV
import importlib.util
import warnings 
warnings.filterwarnings('ignore')
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning,NumbaExperimentalFeatureWarning, NumbaWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaExperimentalFeatureWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

from numba import njit
from numba.typed import List
import numba, os, sys
import numpy as np
from setup import make
game_name = 'MachiKoro'
env = make(game_name)
getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
run = env.run

A:\ENV


In [7]:
def run_game(game_name):
    path  = f'A:\ENV\Agent\Ifelse\{game_name}'
    env = make(game_name)
    def load_module_player(player, game_name = None):
        spec = importlib.util.spec_from_file_location('Agent_player', f"Agent/Ifelse/{game_name}/{player}.py")
        module = importlib.util.module_from_spec(spec)
        sys.modules[spec.name] = module
        spec.loader.exec_module(module)
        return module

    for i in os.listdir(path):
        if '.py' in i:
            path_new = path + '/' + i
            #  print(path_new)
            agent = load_module_player(i[:-3], game_name = game_name)
            win, per = env.run(agent.Test, 10000, agent.DataAgent(), 1)
            print(f'game name: {game_name}, Agent: {i[:-3]} | Win rate: {win}/1')

#  for game_name in ['Exploding_Kitten', 'Fantan', 'GoFish', 'WelcomeToTheDungeon_v1', 'WelcomeToTheDungeon_v2', 'CatanNoExchange', ]:

for game_name in ['TicketToRide']:
    run_game(game_name) 

game name: TicketToRide, Agent: PT | Win rate: 98/1


#  Graphic


In [1]:
%cd A:/ENV

A:\ENV


In [9]:
from setup import make
from src.Base.MachiKoro.env import bot_lv0
env = make("Durak")

In [ ]:
env.render(Agent="human", per_data=[0], level=0, max_temp_frame=100)

In [ ]:
import time
for i in range(10):
    a = time.time()
    env.render(Agent=bot_lv0, per_data=[0], level=1, max_temp_frame=100)
    b = time.time()
    print(int(b-a), i)
    clear_output(wait=True)

# Remove file

In [2]:
%cd ..

a:\ENV


In [15]:
import os, shutil

In [17]:
for agent in os.listdir('A:/ENV/src/Agent'):
    try:
        list_game = os.listdir(f'A:/ENV/src/Agent/{agent}/Data/')
        for game in list_game:
            if 'CatanNoExchange_1' in game:
                print(f'A:/ENV/src/Agent/{agent}/Data/{game}')
                # os.remove(f'A:/ENV/src/Agent/{agent}/Data/{game}')
                shutil.rmtree(f'A:/ENV/src/Agent/{agent}/Data/{game}')
    except: pass

In [29]:
%cd ..
import os
path = 'A:/ENV/src/Agent/'

a:\


In [21]:
import os
path = 'A:/ENV/src/Agent/'

for agent in os.listdir(path):
    try:
        for file in os.listdir(f'{path}{agent}/Data'):
            if '0' in f'{path}{agent}/Data/{file}':
                path_old = f'{path}{agent}/Data/{file}'
                path_new = path_old.replace('0', '1')
                # os.rename(path_old, path_new)
                print(path_old, path_new)
    except: pass

# Kiểm tra level 1

In [1]:
%cd ..

a:\ENV


In [4]:
import os
from tests.CheckEnv import check_pytest
for env_name in os.listdir('src/Base'):
    try:
        check = check_pytest(env_name)
        print(env_name, check)
    except: 
        print(env_name, 'error')

Catan (True, [])
CatanNoExchange (False, ['run, Số trận thắng khi kết thúc khác với sô trận check. Thắng khi dùng getReward: 2508, win: 2507'])
Century (False, ['hàm run không train được với agent không numba'])
Durak (True, [])
Exploding_Kitten (True, [])
Fantan (True, [])
GoFish (True, [])
Imploding_Kitten (True, [])
MachiKoro (True, [])
Poker (True, [])
Sheriff (True, [])
Splendor (True, [])
Splendor_v2 (True, [])
Splendor_v3 (True, [])
StoneAge (True, [])
SushiGo (True, [])
TicketToRide (False, ['run, Số trận thắng khi kết thúc khác với sô trận check. Thắng khi dùng getReward: 2027, win: 2028'])
TLMN (True, [])
WelcomeToTheDungeon_v1 (True, [])
WelcomeToTheDungeon_v2 (True, [])
__init__.py error
__pycache__ error
